<h1> Problem 7 - 10 001st Prime Number </h1>

By listing the first six prime numbers: $2$, $3$, $5$, $7$, $11$ and $13$. The sixth prime is $13$.

What is the 10001st prime number?

---


The Sieve of Eratosthenes allows me to find all the prime numbers up to a certain number. However, I don't know how many prime numbers there will be up to that number. 

According to [wikipedia](https://en.wikipedia.org/wiki/Prime-counting_function), Gauss and Legendre conjectured that the number of primes less than or equal to $x$ is given by, 
\begin{equation}
    \pi(x) \approx \frac{x}{\ln{x}}
\end{equation}
where $\pi(x)$ is approximately the number of primes less than or equal to $x$. This was proved to be true by other mathematicians.

Using this, I can approximate how large a number is required to find the 10001st prime, 
\begin{equation*}
    \pi(.12 \times 10^6) \approx \frac{.12 \times 10^6}{\ln{(.12 \times 10^6)}} \approx 10261
\end{equation*}

In [2]:
fn seive(n: usize) -> Vec<u8> {
    // Initialise aray of boolean values from 2 to n (shifted in implementation)
    // i.e. index 0 = 2, index 1 = 3, ... etc
    let mut out = vec![1u8; (n-2) as usize];
    
    // Max iteration number = sqrt(n)
    let n_sqrt: usize = ((n as f64).sqrt() + 1.0) as usize;
    
    // For i = 2, ..., sqrt(n)
    for i in 2..n_sqrt {
        // if that index is true
        if out[i-2] == 1u8 {
            // Iterate over j = i^2, i^2 + i, ..., n
            let mut k: usize = 0;
            let mut j: usize = i*i + k*i; 
            while j < n {
                out[j-2] = 0u8;
                k += 1;
                j = i*i + k*i;
            }
        }
    }
    out
}

In [3]:
fn seive_improved(n: usize) -> Vec<u8> {
    // Initialise aray of boolean values from 2 to n (shifted in implementation)
    // i.e. index 0 = 2, index 1 = 3, ... etc
    let mut out = vec![1u8; n-2];
    
    // Max iteration number = sqrt(n)
    let n_sqrt: usize = ((n as f64).sqrt() + 1.0) as usize;
    
    // For i = 2, ..., sqrt(n)
    for i in 2..n_sqrt {
        // if that index is true
        if let Some(1u8) = out.get(i-2) {
            // Iterate over j = i^2, i^2 + i, ..., n
            let max_k = (n - i*i - 1)/i; 
            (0..=max_k).for_each(|k| out[i*i + k*i - 2] = 0u8);
        }
    }
    out
}

In [4]:
fn find_nth_prime(n: u32, initial_guess: u32) -> Option<u32> {
    
    let seive_output: Vec<u32> = seive_improved(initial_guess as usize).iter().map(|&x| x as u32).collect();
    
    let num_primes: u32 = seive_output.iter().sum();
//     println!("{}", num_primes);
    
    let diff: u32 = num_primes - n;
//     println!("{}", diff);
    
    let mut count_primes: u32 = 0;
    
    for i in (0..(initial_guess-2)).rev() {
//         println!("{}", i);
        if seive_output.get(i as usize).unwrap() == &1u32 {
            if diff == count_primes {
                return Some(i + 2);
            }
            count_primes += 1;
        }
    }
    None
}

In [5]:
let soln: u32 = find_nth_prime(10001, 120000).unwrap();
println!("The 10001st prime is {}", soln);

The 10001st prime is 104743



---

<h1> Problem 10 - Summation of Primes </h1>

The sum of primes below 10 is $2+3+5+7 = 17$

Find the sum of all primes below 2 million. 



<h3> The Method </h3>

If I was in Python, I'd use numpy to get the index of where a prime exists (`True`) then sum that array. Or I could dot product it with an array of 1s. 
I don't know how I could do this in Rust, but I know Rust is very efficient for functional programming methods.

*What information do I need?*
 Numbers that are prime 

*What information do I have?*
 From the seive of eratosthenes, I have a boolean vector (in this implementation they are ints of 0 and 1) that tells me whether that $index + 2$ is prime. 

*Therefore?* I can use `iter().enumerate()` it gives me the index (which I can use to find the actual number) then multiply it by the value at that location. If the value at that location is 0, then it is not prime and it is not added to the sum. If the value at that location is 1, then it is prime and it is added to the sum. 
\begin{equation}
    \sum_{i=0}^{n-2} ix
\end{equation}
where $n$ is the value we are summing up to, $i$ is the index of the array and $x$ is $1$ or $0$ to indicate if number is prime. 

Actually, [stack overflow](https://stackoverflow.com/questions/30422848/how-do-i-compute-the-dot-product-of-two-rust-arrays-slices-vectors) show that there is a way in Rust to use map to find the dot product of two arrays 

In [6]:
fn sum_primes(n: u32) -> u64{
    // Find all primes up to n using the seive of eratosthenes
    let seive_output: Vec<u64> = seive(n as usize).iter().map(|&x| x as u64).collect();
    // Initialise output sum as 0 
    let mut out_sum: u64 = 0; 
    
    // for_each as we want the side effects (idiomatic way of writing map)
    // In this case, the side effects is the adding to the output sum 
    seive_output.iter().enumerate().for_each(|(i, x)| {out_sum += (i as u64 + 2)*(*x)});
    out_sum
}

In [7]:
let soln: u64 = sum_primes(2000000);
println!("The sum of the first 2 million primes is {}", soln);

The sum of the first 2 million primes is 142913828922



---


In [13]:
let seive_output = seive_improved(5001 as usize).iter()
                                                .enumerate()
                                                .filter(|(counter, &x)| &x == &1u8)
                                                .map(|(counter, _)| counter + 2 ).collect::<Vec<usize>>();
println!("{:?}", seive_output);

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 12

In [15]:
println!("{}", 4999*4999 + 283*283*283 + 67*67*67*67 - 50*1000000);

17806309
